# Predicting world happiness

The following code sets up the libraries and creates cleaned and pre-processed training, validation and test data that we will use in this document.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.preprocessing import OneHotEncoder, scale, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
import statsmodels.api as sm
from itertools import product
from joblib import Parallel, delayed

from functions.clean_happiness import clean_happiness
from functions.split_data import split_data

pd.set_option('display.max_columns', None)

In [2]:
# load the data
happiness_orig = pd.read_excel("../data/WHR2018Chapter2OnlineData.xls", sheet_name=0)

In [3]:
happiness_orig.columns

Index(['country', 'year', 'Life Ladder', 'Log GDP per capita',
       'Social support', 'Healthy life expectancy at birth',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption', 'Positive affect', 'Negative affect',
       'Confidence in national government', 'Democratic Quality',
       'Delivery Quality', 'Standard deviation of ladder by country-year',
       'Standard deviation/Mean of ladder by country-year',
       'GINI index (World Bank estimate)',
       'GINI index (World Bank estimate), average 2000-15',
       'gini of household income reported in Gallup, by wp5-year'],
      dtype='object')

Since all features, except for `year` and `country`, can be used to describe the development of the corresponding `country`, we do not consider grouping by country when splitting the dataset.

Considering that the `year` variable in the dataset is a relatively complete time variable, and as the analysis in `02_eda.ipynb` shows, the overall trend of happiness data does not exhibit significant fluctuations over time, we chose to split the dataset based on time.

The splitting function `split_data()` is stored in the file `functions/split_data.py`. Below is the code for the `split_data()` function.

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def split_data(df):
    # Check if the 'year' column exists
    if 'year' not in df.columns:
        raise ValueError("Dataframe must contain a 'year' column")

    # Sort by the 'year' column in ascending order
    df_sorted = df.sort_values(by='year')

    # Obtain all years and split them into three subsets with a ratio of 6:1:3
    unique_years = sorted(df['year'].unique())
    n_years = len(unique_years)
    train_years = unique_years[:int(0.6 * n_years)]
    val_years = unique_years[int(0.6 * n_years):int(0.7 * n_years)]
    test_years = unique_years[int(0.7 * n_years):]

    # Group the data by year
    train = df_sorted[df_sorted['year'].isin(train_years)]
    val = df_sorted[df_sorted['year'].isin(val_years)]
    test = df_sorted[df_sorted['year'].isin(test_years)]

    # Output the three datasets
    return train, val, test


The cleaned data has been split into the following:

In [5]:
happiness_train,happiness_val,happiness_test = split_data(happiness_orig)

In [6]:
happiness_train

,country,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Confidence in national government,Democratic Quality,Delivery Quality,Standard deviation of ladder by country-year,Standard deviation/Mean of ladder by country-year,GINI index (World Bank estimate),"GINI index (World Bank estimate), average 2000-15","gini of household income reported in Gallup, by wp5-year"
1141,Romania,2005,5.048648,9.592577,0.837685,64.108208,0.800121,NaN,0.956885,0.642016,0.345687,0.272887,0.246735,-0.113104,2.281933,0.451989,0.298,0.290929,NaN
1405,Turkey,2005,4.718734,9.699514,0.819936,62.788372,0.623115,NaN,0.876999,0.556581,NaN,0.595774,-0.300653,0.140000,2.342795,0.496488,0.426,0.403077,NaN
177,Brazil,2005,6.636771,9.417304,0.882923,62.258163,0.882186,NaN,0.744994,0.818337,0.301780,0.340625,0.096596,-0.156601,2.436181,0.367073,0.566,0.550214,NaN
656,Italy,2005,6.853784,10.534875,0.928001,71.050049,0.802195,NaN,0.943912,0.678837,0.294698,0.238738,0.773581,0.622240,1.884019,0.274887,0.338,0.341273,NaN
120,Belgium,2005,7.262290,10.591697,0.934875,70.095184,0.923843,NaN,0.597554,0.796279,0.260380,0.551705,1.108286,1.409674,1.499964,0.206541,0.293,0.285182,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,Benin,2011,3.870280,7.507083,0.477494,50.454754,0.772919,-0.116167,0.849472,0.625820,0.218678,0.777150,0.242373,-0.537246,1.694988,0.437950,0.434,0.432667,0.355524
26,Angola,2011,5.589001,8.684613,0.723094,50.894405,0.583702,0.064475,0.911320,0.658647,0.361063,0.232387,-0.747358,-1.215250,2.212474,0.395862,NaN,0.427000,NaN
544,Haiti,2011,4.844574,7.353901,0.567039,51.710850,0.412588,0.246202,0.681960,0.625240,0.244856,0.456668,-0.869596,-1.339227,1.731062,0.357320,NaN,0.409000,0.499586
1488,Uruguay,2011,6.554047,9.792821,0.891282,67.595703,0.851442,-0.093564,0.556286,0.805346,0.252250,0.631277,1.039264,0.741918,2.172416,0.331462,0.434,0.442600,0.421388


In [7]:
happiness_val

,country,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Confidence in national government,Democratic Quality,Delivery Quality,Standard deviation of ladder by country-year,Standard deviation/Mean of ladder by country-year,GINI index (World Bank estimate),"GINI index (World Bank estimate), average 2000-15","gini of household income reported in Gallup, by wp5-year"
1158,Russia,2012,5.620736,10.121795,0.901295,61.947884,0.609104,-0.307607,0.937518,0.611164,0.173604,0.452002,-0.898265,-0.654854,2.080930,0.370224,0.407,0.399250,0.376690
827,Macedonia,2012,4.639647,9.354403,0.798305,65.123634,0.613056,-0.086593,0.919845,0.641887,0.421752,0.369647,-0.260573,0.005125,2.174477,0.468673,0.401,0.390167,0.375651
1335,Syria,2012,3.164491,8.609065,0.588395,61.474083,0.466771,0.303708,0.672964,0.464439,0.704590,NaN,-2.263247,-1.254518,2.508209,0.792611,NaN,0.358000,0.458947
836,Madagascar,2012,3.550610,7.223259,0.673088,55.214088,0.487008,-0.047278,0.853590,0.689856,0.193977,0.350055,-0.709704,-0.805915,1.543136,0.434612,0.427,0.431000,0.536564
1342,Taiwan Province of China,2012,6.125917,NaN,0.825072,70.059998,0.698195,NaN,0.802829,0.821362,0.140011,0.335142,0.870516,1.028489,2.035462,0.332271,NaN,NaN,0.433278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,Ghana,2013,4.965053,8.244684,0.676289,53.843502,0.793794,-0.057976,0.880178,0.690766,0.210819,0.485888,0.249730,0.009688,1.828295,0.368233,NaN,0.425000,0.544950
743,Kuwait,2013,6.480031,11.212962,0.861948,64.947571,0.750525,NaN,NaN,0.752348,0.282629,NaN,-0.245775,0.008790,2.074005,0.320061,NaN,NaN,0.431248
244,Canada,2013,7.593794,10.653385,0.936239,71.799850,0.916014,0.301623,0.406236,0.851297,0.262850,0.505976,1.257454,1.792711,1.658059,0.218344,0.340,0.336800,0.552203
481,Georgia,2013,4.348921,9.018455,0.559166,63.856758,0.722128,-0.240374,0.348714,0.595041,0.199907,0.633563,-0.151276,0.451273,1.876835,0.431563,0.400,0.403438,0.442743


In [8]:
happiness_test

,country,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Confidence in national government,Democratic Quality,Delivery Quality,Standard deviation of ladder by country-year,Standard deviation/Mean of ladder by country-year,GINI index (World Bank estimate),"GINI index (World Bank estimate), average 2000-15","gini of household income reported in Gallup, by wp5-year"
143,Bhutan,2014,4.938578,8.904688,0.880342,60.230301,0.834222,0.275819,0.650338,0.858864,0.324098,0.958492,0.482543,0.229389,0.991039,0.200673,NaN,0.412333,0.471104
1193,Senegal,2014,4.394777,7.704610,0.855522,56.940323,0.692353,-0.030414,0.699660,0.724697,0.157227,0.664996,0.047028,-0.162450,1.374460,0.312749,NaN,0.402333,0.391568
1243,Slovenia,2014,5.678395,10.254766,0.908348,70.943291,0.887748,0.044557,0.909118,0.619818,0.290812,0.176851,0.960940,0.848770,2.046137,0.360337,0.257,0.249091,0.378872
1052,Pakistan,2014,5.435658,8.428630,0.551683,56.990078,0.543139,0.126021,0.676928,0.584937,0.295480,0.425877,-1.558652,-0.758009,2.169976,0.399211,NaN,0.312571,0.434137
1132,Portugal,2014,5.126912,10.166763,0.861829,71.106895,0.846810,-0.133560,0.941070,0.705072,0.357692,0.227394,0.956860,0.955930,2.364470,0.461188,0.356,0.367000,0.440264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,Namibia,2017,4.441306,9.203362,0.828339,55.495426,0.810402,-0.201766,0.831303,0.720678,0.277252,0.745854,NaN,NaN,2.739773,0.616885,NaN,0.621500,0.602736
974,Nepal,2017,4.736692,7.801309,0.816383,61.265957,0.845148,0.146511,0.770177,0.570577,0.375978,0.559579,NaN,NaN,2.468454,0.521134,NaN,0.383000,0.463747
320,Congo (Kinshasa),2017,4.311033,6.625341,0.669688,50.788219,0.704240,0.095793,0.809182,0.550526,0.404262,0.478412,NaN,NaN,2.642479,0.612957,NaN,0.421500,0.632452
673,Ivory Coast,2017,5.037735,8.180943,0.661375,46.989941,0.732098,-0.095354,0.770940,0.697735,0.357456,0.516654,NaN,NaN,2.612090,0.518505,NaN,0.420667,0.557954


Next, we will perform data cleaning on the training and validation datasets.

In [9]:
# clean the data
happiness_train_clean = clean_happiness(happiness_train,impute_method="average")
happiness_val_clean = clean_happiness(happiness_val,impute_method="average")

In [10]:
happiness_train_clean.columns

Index(['year', 'happiness', 'log_gdp_per_capita', 'social_support',
       'life_expectancy', 'freedom_choices', 'generosity', 'corruption',
       'positive_affect', 'negative_affect', 'government_confidence',
       'democratic_quality', 'delivery_quality', 'sd_ladder',
       'sd_d_mean_ladder', 'gini_wb_estimate', 'gini_wb_estimate_average',
       'gini_hh_income'],
      dtype='object')

In [11]:
happiness_train_clean

,year,happiness,log_gdp_per_capita,social_support,life_expectancy,freedom_choices,generosity,corruption,positive_affect,negative_affect,government_confidence,democratic_quality,delivery_quality,sd_ladder,sd_d_mean_ladder,gini_wb_estimate,gini_wb_estimate_average,gini_hh_income
0,2005,3.723590,7.168690,0.450662,49.209663,0.718114,0.181819,0.881686,0.517637,0.258195,0.612072,-1.929690,-1.655084,1.774662,0.476600,0.000,0.000,0.441906
1,2006,3.723590,7.168690,0.450662,49.209663,0.718114,0.181819,0.881686,0.517637,0.258195,0.612072,-1.929690,-1.655084,1.774662,0.476600,0.000,0.000,0.441906
2,2007,3.723590,7.168690,0.450662,49.209663,0.718114,0.181819,0.881686,0.517637,0.258195,0.612072,-1.929690,-1.655084,1.774662,0.476600,0.000,0.000,0.441906
3,2008,3.723590,7.168690,0.450662,49.209663,0.718114,0.181819,0.881686,0.517637,0.258195,0.612072,-1.929690,-1.655084,1.774662,0.476600,0.000,0.000,0.441906
4,2009,4.401778,7.333790,0.552308,49.624432,0.678896,0.203614,0.850035,0.583926,0.237092,0.611545,-2.044093,-1.635025,1.722688,0.391362,0.000,0.000,0.441906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,2007,3.280247,7.313939,0.828113,40.244083,0.455957,-0.058068,0.946287,0.660861,0.264989,0.225752,-1.340245,-1.653740,1.929571,0.588240,0.432,0.432,0.545112
1088,2008,3.174264,7.102516,0.843475,41.607285,0.343556,-0.063873,0.963846,0.630983,0.250060,0.181594,-1.381488,-1.701545,1.685007,0.530834,0.432,0.432,0.545112
1089,2009,4.055914,7.197595,0.805781,43.110626,0.411089,-0.051992,0.930818,0.735503,0.218419,0.285287,-1.353181,-1.717821,2.024098,0.499048,0.432,0.432,0.545112
1090,2010,4.681570,7.296330,0.856638,44.672630,0.664718,-0.062908,0.828361,0.747702,0.122150,0.471201,-1.289599,-1.693678,1.511014,0.322758,0.432,0.432,0.680030


In [12]:
happiness_val_clean

,year,happiness,log_gdp_per_capita,social_support,life_expectancy,freedom_choices,generosity,corruption,positive_affect,negative_affect,government_confidence,democratic_quality,delivery_quality,sd_ladder,sd_d_mean_ladder,gini_wb_estimate,gini_wb_estimate_average,gini_hh_income
0,2012,3.782938,7.517126,0.520637,50.709263,0.530935,0.247159,0.775620,0.710385,0.267919,0.435440,-1.842996,-1.404078,1.798283,0.475367,0.00,0.00000,0.344540
1,2013,3.572100,7.503376,0.483552,51.042980,0.577955,0.074735,0.823204,0.620585,0.273328,0.482847,-1.879709,-1.403036,1.223690,0.342569,0.00,0.00000,0.304368
2,2012,5.510124,9.246649,0.784502,68.028885,0.601512,-0.174559,0.847675,0.606636,0.271393,0.364894,-0.060784,-0.328862,1.921203,0.348668,0.29,0.30325,0.568153
3,2013,4.550648,9.258439,0.759477,68.291374,0.631830,-0.132977,0.862905,0.633609,0.338379,0.338095,0.070411,-0.330956,2.315580,0.508846,0.29,0.30325,0.633796
4,2012,5.604596,9.485086,0.839397,64.739365,0.586663,-0.198871,0.690116,0.604023,0.229716,0.000000,-1.115535,-0.771172,1.939237,0.346008,0.00,0.27600,0.421409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,2013,4.217679,8.261730,0.693905,54.233780,0.542547,-0.187269,0.885197,0.558500,0.265685,0.387677,-1.854220,-1.092611,2.295311,0.544212,0.00,0.35700,0.429792
292,2012,5.013375,8.163204,0.780023,50.674156,0.787760,-0.002135,0.806394,0.725965,0.250368,0.594114,0.264068,-0.388092,2.200456,0.438917,0.00,0.52740,0.610944
293,2013,5.243996,8.182191,0.761312,51.510342,0.769912,-0.114406,0.732268,0.734979,0.307960,0.552761,0.164946,-0.385220,2.562254,0.488607,0.00,0.52740,0.514960
294,2012,4.955101,7.534424,0.896476,47.654789,0.469531,-0.072877,0.858691,0.669279,0.177311,0.407084,-1.125315,-1.555728,1.853195,0.373997,0.00,0.43200,0.487203


## LS with multiple predictors

Since the `LinearRegression()` function from sklearn does not compute coefficient statistics, we will use the statsmodels package to fit the model using the OLS function instead.

Based on the `02_eda.ipynb` file, we select the four features most strongly correlated with `happiness` for linear regression fitting.

In [13]:
y = happiness_train_clean['happiness']
X_multi4 = happiness_train_clean[['social_support', 'life_expectancy', 'democratic_quality', 'sd_d_mean_ladder']]
X_multi4 = sm.add_constant(X_multi4)

ls_multi4 = sm.OLS(y, X_multi4).fit()


### Comparing coefficients

Comparing the coefficients using the *theoretical* $t$-values can be done by applying the `summary()` method to the `ls_multi4` object (look at the `t value` column): 

In [14]:
ls_multi4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              happiness   R-squared:                       0.749
Model:                            OLS   Adj. R-squared:                  0.748
Method:                 Least Squares   F-statistic:                     810.9
Date:                Thu, 02 Jan 2025   Prob (F-statistic):               0.00
Time:                        20:26:49   Log-Likelihood:                -914.05
No. Observations:                1092   AIC:                             1838.
Df Residuals:                    1087   BIC:                             1863.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  4.8647      0.201     24.227      0.000       4.471       5.259
social_support         1.0621      0.145      7.314      0.000       0.777       1.347
life_expectancy        0.0323      0.002     15.594      0.000       0.028       0.036
democratic_quality     0.2358      0.024      9.658      0.000       0.188       0.284
sd_d_mean_ladder      -6.1486      0.226    -27.239      0.000      -6.592      -5.706
==============================================================================
Omnibus:                       24.942   Durbin-Watson:                   0.410
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               26.464
Skew:                           0.347   Prob(JB):                     1.79e-06
Kurtosis:                       3.314   Cond. No.                     1.03e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.03e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""


However, in veridical data science we prefer to empirically estimate the standard deviation of the coefficients in order to compute the t-values, where $t_j = b_j / SD(b_j)$, e.g., using the bootstrap. 

The following code uses the bootstrap (sampling with replacement) to create 1000 different perturbed versions of the data, computes a LS fit for each bootstrap sample, and reports the bootstrap coefficients in a nice tidy data frame (tibble):

In [15]:
# use the bootstrap (sampling with replacement) to create 1000 different perturbed versions of the data, computes a LS fit for each bootstrap sample, and reports the bootstrap coefficients in a nice tidy data frame

# define the number of bootstrap samples
n_bootstraps = 1000

# define the number of observations in the original dataset
n = happiness_train_clean.shape[0]

# create an empty list to store the bootstrap samples
bootstrap_samples = []

# create an empty list to store the bootstrap coefficients
bootstrap_coefs = []

# create a for loop to generate the bootstrap samples
for i in range(n_bootstraps):
    bootstrap_sample = happiness_train_clean.sample(n=n, replace=True)
    bootstrap_samples.append(bootstrap_sample)
    y_boot = bootstrap_sample['happiness']
    X_multi4_boot = bootstrap_sample[['social_support', 'life_expectancy', 'democratic_quality', 'sd_d_mean_ladder']]
    X_multi4_boot = sm.add_constant(X_multi4_boot)
    ls_multi4_boot = sm.OLS(y_boot, X_multi4_boot).fit()
    bootstrap_coefs.append(ls_multi4_boot.params)

# convert the bootstrap coefficients into a data frame
bootstrap_coefs_df = pd.DataFrame(bootstrap_coefs)

bootstrap_coefs_df

,const,social_support,life_expectancy,democratic_quality,sd_d_mean_ladder
0,4.195460,0.921521,0.043122,0.220068,-5.847771
1,5.143976,0.864264,0.031805,0.237971,-6.365234
2,5.267854,0.929439,0.028004,0.273902,-6.338619
3,4.531503,1.144483,0.036319,0.187118,-6.090270
4,4.772100,0.993824,0.035013,0.192805,-6.182998
...,...,...,...,...,...
995,5.119954,1.132273,0.028291,0.248136,-6.334003
996,4.407039,1.459374,0.032366,0.225207,-5.801857
997,4.574894,1.244776,0.033533,0.205108,-5.980676
998,4.516782,1.531115,0.031162,0.229821,-6.013192



Then we can compute the standard deviation of each of the bootstrap coefficients, and use these standard deviation estimates to compute the standardized coefficients.

In [16]:
# compute the standard deviation of each of the bootstrap coefficients, 
# and use these standard deviation estimates to compute the standardized coefficients

# compute the standard deviation of each of the bootstrap coefficients
bootstrap_coefs_std = bootstrap_coefs_df.std(axis=0)

# compute the standardized coefficients
ls_multi4.summary2().tables[1]['Coef.'] / bootstrap_coefs_std

const                 14.514360
social_support         3.489385
life_expectancy        8.046714
democratic_quality     8.807645
sd_d_mean_ladder     -21.017900
dtype: float64

## Ridge and lasso

### Choosing $\lambda$ using cross-validation

Using 10-fold cross-validation to choose the ridge hyperparameter can be done using the `cross_validate()` function from the scikit.learn library.


In [17]:
X = happiness_train_clean.drop(columns=['happiness'])
# scale the predictors
X_std = (X - X.mean()) / X.std()
y = happiness_train_clean['happiness']

In [18]:
# use 10-fold cross-validation to select the best lambda (alpha) value for the ridge regression model

# define the alpha values to test
# note that the start/stop values in the first two arguments are the exponents
alphas = np.logspace(-1, 6, 100)

# create an empty list to store the cross-validation scores
ridge_cv_scores = []

# create a for loop to compute the cross-validation score for each alpha value
for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge_cv = cross_validate(estimator=ridge,
                              X=X_std,
                              y=y,
                              cv=10,
                              scoring='neg_root_mean_squared_error')
    ridge_cv_scores.append({'alpha': alpha,
                            'log_alpha': np.log(alpha),
                            'test_mse': -np.mean(ridge_cv['test_score'])})

# convert the cross-validation scores into a data frame
ridge_cv_scores_df = pd.DataFrame(ridge_cv_scores)

# plot the cross-validation scores as a function of alpha
px.line(ridge_cv_scores_df,
        x='log_alpha',
        y='test_mse',
        title='Ridge')

In [19]:
# use 10-fold cross-validation to select the best lambda (alpha) value for the lasso regression model

# define the alpha values to test
# note that the start/stop values in the first two arguments are the exponents
alphas = np.logspace(-1, 4, 100)

# create an empty list to store the cross-validation scores
lasso_cv_scores = []

# create a for loop to compute the cross-validation score for each alpha value
for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso_cv = cross_validate(estimator=lasso,
                              X=X_std,
                              y=y,
                              cv=10,
                              scoring='neg_root_mean_squared_error')
    lasso_cv_scores.append({'alpha': alpha,
                            'log_alpha': np.log(alpha),
                            'test_mse': -np.mean(lasso_cv['test_score'])})

# convert the cross-validation scores into a data frame
lasso_cv_scores_df = pd.DataFrame(lasso_cv_scores)

# plot the cross-validation scores as a function of alpha
px.line(lasso_cv_scores_df,
        x='log_alpha',
        y='test_mse',
        title='Lasso')



### Fitting ridge and lasso models

Let's extract the "best" alpha values for each fit:

In [20]:
# identify the value of alpha that minimizes the cross-validation score for ridge
ridge_alpha_min = ridge_cv_scores_df.sort_values(by='test_mse').head(1).alpha.values[0]
# compute the min MSE and the SE of the MSE
mse_se_ridge = ridge_cv_scores_df['test_mse'].std() / np.sqrt(10)
mse_min_ridge = ridge_cv_scores_df['test_mse'].min()


# identify the value of alpha that minimizes the cross-validation score for ridge within 1SE
ridge_alpha_1se = ridge_cv_scores_df[(ridge_cv_scores_df['test_mse'] <= mse_min_ridge + mse_se_ridge) & 
                                     (ridge_cv_scores_df['test_mse'] >= mse_min_ridge - mse_se_ridge)].sort_values(by='alpha', ascending=False).head(1).alpha.values[0]


# identify the value of alpha that minimizes the cross-validation score for lasso
lasso_alpha_min = lasso_cv_scores_df.sort_values(by='test_mse').head(1).alpha.values[0]
# compute the min MSE and the SE of the MSE
mse_se_lasso = lasso_cv_scores_df['test_mse'].std() / np.sqrt(10)
mse_min_lasso = lasso_cv_scores_df['test_mse'].min()

# identify the value of alpha that minimizes the cross-validation score for lasso within 1SE
lasso_alpha_1se = lasso_cv_scores_df[(lasso_cv_scores_df['test_mse'] <= mse_min_lasso + mse_se_lasso) & 
                                     (lasso_cv_scores_df['test_mse'] >= mse_min_lasso - mse_se_lasso)].sort_values(by='alpha', ascending=False).head(1).alpha.values[0]


In [21]:
print('Ridge (min): ', ridge_alpha_min)
print('Ridge (1SE): ', ridge_alpha_1se)
print('Lasso (min): ', lasso_alpha_min)
print('Lasso (1SE): ', lasso_alpha_1se)

Ridge (min):  0.43287612810830584
Ridge (1SE):  93.26033468832199
Lasso (min):  0.1
Lasso (1SE):  0.12618568830660204


Having chosen the `alpha` values for ridge and lasso using CV, we can fit the ridge and lasso models.

In [22]:
# use ridge_alpha_min to fit the ridge regression model
ridge_min_fit = Ridge(alpha=ridge_alpha_min).fit(X=X_std, y=y)
ridge_1se_fit = Ridge(alpha=ridge_alpha_1se).fit(X=X_std, y=y)

# use lasso_alpha_min to fit the lasso regression model
lasso_min_fit = Lasso(alpha=lasso_alpha_min).fit(X=X_std, y=y)
lasso_1se_fit = Lasso(alpha=lasso_alpha_1se).fit(X=X_std, y=y)



### Exploring coefficient shrinkage

Let's visualize how much each regularization algorithm shrinks each coefficient.

To do that, we need to extract the coefficients from each regularized fit, as well as the original fit with all features. 

First, we will re-compute the original fit using standardized variables so that the coefficients are comparable to the ridge and lasso coefficients):

In [23]:
# compute the LS model using linearregression (X has been standardized above)
ls_std = LinearRegression()
ls_std.fit(X=X_std, y=y)

# extract the coefficients from the LS model
ls_std_coefs = pd.DataFrame([round(abs(x), 2) for x in ls_std.coef_],
                            index=list(happiness_train_clean.drop(columns=['happiness']).columns),
                            columns=['ls'])

Then we can extract the coefficients from each of the regularized fits.

In [24]:
# extract the coefficients from the lasso and ridge models
ridge_min_coefs = pd.DataFrame([round(abs(x), 2) for x in ridge_min_fit.coef_],
                           index=list(happiness_train_clean.drop(columns=['happiness']).columns),
                           columns=['ridge_min'])
ridge_1se_coefs = pd.DataFrame([round(abs(x), 2) for x in ridge_1se_fit.coef_],
                           index=list(happiness_train_clean.drop(columns=['happiness']).columns),
                           columns=['ridge_1se'])
lasso_min_coefs = pd.DataFrame([round(abs(x), 2) for x in lasso_min_fit.coef_],
                            index=list(happiness_train_clean.drop(columns=['happiness']).columns),
                            columns=['lasso_min'])
lasso_1se_coefs = pd.DataFrame([round(abs(x), 2) for x in lasso_1se_fit.coef_],
                            index=list(happiness_train_clean.drop(columns=['happiness']).columns),
                            columns=['lasso_1se'])


In [25]:
# compute the order of the coeffiicnets in terms of absolute value
ls_std_coefs = ls_std_coefs.sort_values(by='ls', ascending=False)
# merge the coefficients into a single data frame
coefs = ls_std_coefs.merge(ridge_min_coefs, left_index=True, right_index=True)
coefs = coefs.merge(lasso_min_coefs, left_index=True, right_index=True) \
             .merge(ridge_1se_coefs, left_index=True, right_index=True) \
             .merge(lasso_1se_coefs, left_index=True, right_index=True)
# convert to long form 
coefs = coefs.reset_index().melt(id_vars='index', var_name='model', value_name='coefficients')
coefs = coefs.rename(columns={'index': 'variable'})

# place the LS coefficients in its own column and remove the original rows with LS coefficients
coefs["ls_coefficients"] = list(coefs.query('model == "ls"').coefficients) * 5
coefs = coefs.query('model != "ls"')


Then can create a plot that compares the unregularized and regularized coefficients for each regularized fit.

The figure below shows that the lasso fit with $\lambda_{min}$ implemented almost no shrinkage (implying that this regularization parameter was not large enough to actually perform any meaningful regularization). The ridge fit with $\lambda_{min}$ involves slightly more regularization than the corresponding lasso fit with $\lambda_{min}$, but most of the coeﬀicients are still very similar to their unregularized counterparts. 

In [26]:
fig = px.scatter(x='variable', 
                 y='ls_coefficients', 
                 labels=dict(variable='Variable', ls_coefficients='Coefficients'),
                 data_frame=coefs, 
                 facet_row='model',
                 color_discrete_sequence=['rgb(128,128,128)'],
                 height=900)

# add just the ls_coefficients values to each facet in fig
i = 5
for model_var in coefs.model.unique():
    fig.add_trace(go.Scatter(x=coefs.query('model == @model_var').variable,
                             y=coefs.query('model == @model_var').coefficients,
                             mode='markers+lines',
                             line=dict(color='black'),
                             showlegend=False),
                  row=i-1, col=1)
    i-=1

fig

## RF,Bagging and Gradient boosting

We can fit a RF, Bagging and Gradient Boosting algorithm using the entire training dataset and all available features using the `()` function from the sklearn.

In [27]:
x_happiness_train = happiness_train_clean.drop(columns='happiness')
y_happiness_train = happiness_train_clean['happiness']

happiness_rf = RandomForestRegressor(random_state=764)
happiness_rf.fit(happiness_train_clean.drop(columns='happiness'), y_happiness_train)

RandomForestRegressor(random_state=764)

In [28]:
# Define the decision tree regressor
decision_tree = DecisionTreeRegressor(random_state=764)

# Create the Bagging regressor using the decision tree as the base estimator
happiness_bag = BaggingRegressor(base_estimator=decision_tree,
                                 n_estimators=100,  # Number of decision trees in the ensemble
                                 random_state=764)

# Fit the Bagging model on the training data
happiness_bag.fit(happiness_train_clean.drop(columns='happiness'), y_happiness_train)


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



BaggingRegressor(base_estimator=DecisionTreeRegressor(random_state=764),
                 n_estimators=100, random_state=764)

In [29]:
# Create the Bagging regressor using the decision tree as the base estimator
happiness_gb = GradientBoostingRegressor(n_estimators=100,  # Number of boosting stages (trees)
                                     learning_rate=0.1,  # Step size shrinking to prevent overfitting
                                     max_depth=3,  # Maximum depth of the individual trees
                                     random_state=764)

# Fit the Bagging model on the training data
happiness_gb.fit(happiness_train_clean.drop(columns='happiness'), y_happiness_train)


GradientBoostingRegressor(random_state=764)

### RF variable importance


By default, `RandomForestRegressor()` computes the "impurity" importance measure, but it reports it in terms of the "mean decrease in impurity" . 


In [30]:
impurity_importance_df = pd.DataFrame({
    'variable': x_happiness_train.columns,
    'importance': happiness_rf.feature_importances_
}).sort_values('importance')
impurity_importance_df

,variable,importance
0,year,0.000893
16,gini_hh_income,0.002122
5,generosity,0.002670
11,delivery_quality,0.003067
6,corruption,0.003252
8,negative_affect,0.003409
9,government_confidence,0.004478
10,democratic_quality,0.004917
4,freedom_choices,0.007014
15,gini_wb_estimate_average,0.009815


To compute the permutation importance, we can use the `permutation_importance()` function

In [ ]:
permutation_importance_df = pd.DataFrame({
    'variable': x_happiness_train.columns,
    'importance': permutation_importance(happiness_rf, 
                                          x_happiness_train, 
                                          y_happiness_train, 
                                          n_repeats = 10, 
                                          random_state = 0).importances_mean
}).sort_values('importance')
permutation_importance_df

,variable,importance
0,year,0.000413
16,gini_hh_income,0.000632
5,generosity,0.001050
8,negative_affect,0.001137
11,delivery_quality,0.001352
6,corruption,0.001394
10,democratic_quality,0.002230
9,government_confidence,0.002574
4,freedom_choices,0.002944
15,gini_wb_estimate_average,0.003306



And we can visualize these importance measures using a bar plot:

In [32]:
px.bar(permutation_importance_df, 
       y='variable', 
       x='importance', 
       orientation='h', 
       title='Permutation Importance',
       height=1000)

In [33]:
px.bar(impurity_importance_df, 
       y='variable', 
       x='importance', 
       orientation='h', 
       title='Impurity Importance',
       height=1000)


## PCS evaluations


### Predictability

In [34]:
happiness_test_clean=clean_happiness(happiness_test,impute_method="average")


Before we get started, we need to create a matrix version of the test set for fitting ridge and lasso. However, note that we will use the mean and SD from the training set to standardize the test set (the logic here is that if we were using these algorithm to predict the response of a new data point, we would use the training data mean and SD to standardize it).


In [35]:
X_train = happiness_train_clean.drop(columns=['happiness'])
X_test = happiness_test_clean.drop(columns=['happiness'])

# standardize the training and test sets using the mean and SD from the training data
happiness_train_clean_std = (happiness_train_clean - happiness_train_clean.mean()) / happiness_train_clean.std()
happiness_test_clean_std = (happiness_test_clean - happiness_train_clean.mean()) / happiness_train_clean.std()
# extract the predictors and response from the standardized training and test sets
X_train_std = happiness_train_clean_std.drop(columns=['happiness'])
y_test_std = happiness_test_clean_std['happiness']
X_test_std = happiness_test_clean_std.drop(columns=['happiness']) 

Let's compute predictions for each fit (ls_multi_fit, ridge_1se_fit, lasso_1se_fit). Let's re-define the basic LS fits using the standardized data we just defined

In [36]:
ls_multi_fit = LinearRegression()
ls_multi_fit.fit(X=X_train_std[['social_support', 'life_expectancy', 'democratic_quality', 'sd_d_mean_ladder']],
                 y=happiness_train_clean['happiness'])

LinearRegression()

In [37]:
# compute the predictions on the validaion set for ls_multi_fit, ridge_1se_fit, and lasso_1se_fit
ls_multi_test_pred = ls_multi_fit.predict(X=X_test_std[['social_support', 'life_expectancy', 'democratic_quality', 'sd_d_mean_ladder']])
ridge_1se_test_pred = ridge_1se_fit.predict(X=X_test_std)
lasso_1se_test_pred = lasso_1se_fit.predict(X=X_test_std)

In [38]:
# evaluate the rMSE, MAE and correlation of each fit
# create an empty list to store the results
test_results = []
# compute the rMSE, MAE and correlation for each fit
test_results.append({'model': 'ls_multi',
                    'rmse': np.sqrt(np.mean((happiness_test_clean['happiness'] - ls_multi_test_pred)**2)),
                    'mae': np.mean(abs(happiness_test_clean['happiness'] - ls_multi_test_pred)),
                    'corr': np.corrcoef(happiness_test_clean['happiness'], ls_multi_test_pred)[0, 1]})
test_results.append({'model': 'ridge_1se',
                    'rmse': np.sqrt(np.mean((happiness_test_clean['happiness'] - ridge_1se_test_pred)**2)),
                    'mae': np.mean(abs(happiness_test_clean['happiness'] - ridge_1se_test_pred)),
                    'corr': np.corrcoef(happiness_test_clean['happiness'], ridge_1se_test_pred)[0, 1]})
test_results.append({'model': 'lasso_1se',
                    'rmse': np.sqrt(np.mean((happiness_test_clean['happiness'] - lasso_1se_test_pred)**2)),
                    'mae': np.mean(abs(happiness_test_clean['happiness'] - lasso_1se_test_pred)),
                    'corr': np.corrcoef(happiness_test_clean['happiness'], lasso_1se_test_pred)[0, 1]})
# convert the results into a data frame
test_results_df = pd.DataFrame(test_results)
test_results_df

,model,rmse,mae,corr
0,ls_multi,0.743754,0.568215,0.869970
1,ridge_1se,0.436612,0.297527,0.944520
2,lasso_1se,0.489214,0.364968,0.930673


In [39]:
rf_test_pred = happiness_rf.predict(happiness_test_clean.drop(columns='happiness'))

test_results.append({'model': 'rf',
                    'rmse': np.sqrt(np.mean((happiness_test_clean['happiness'] - rf_test_pred)**2)),
                    'mae': np.mean(abs(happiness_test_clean['happiness'] - rf_test_pred)),
                    'corr': np.corrcoef(happiness_test_clean['happiness'], rf_test_pred)[0, 1]})

test_results_df = pd.DataFrame(test_results)

In [40]:
bag_test_pred = happiness_bag.predict(happiness_test_clean.drop(columns='happiness'))

test_results.append({'model': 'bagging',
                    'rmse': np.sqrt(np.mean((happiness_test_clean['happiness'] - bag_test_pred)**2)),
                    'mae': np.mean(abs(happiness_test_clean['happiness'] - bag_test_pred)),
                    'corr': np.corrcoef(happiness_test_clean['happiness'], bag_test_pred)[0, 1]})

test_results_df = pd.DataFrame(test_results)

In [41]:
gb_test_pred = happiness_gb.predict(happiness_test_clean.drop(columns='happiness'))

test_results.append({'model': 'gb',
                    'rmse': np.sqrt(np.mean((happiness_test_clean['happiness'] - gb_test_pred)**2)),
                    'mae': np.mean(abs(happiness_test_clean['happiness'] - gb_test_pred)),
                    'corr': np.corrcoef(happiness_test_clean['happiness'], gb_test_pred)[0, 1]})

test_results_df = pd.DataFrame(test_results)

#### Summary of Prediction Results

In [42]:
test_results_df

,model,rmse,mae,corr
0,ls_multi,0.743754,0.568215,0.869970
1,ridge_1se,0.436612,0.297527,0.944520
2,lasso_1se,0.489214,0.364968,0.930673
3,rf,0.364071,0.263737,0.950452
4,bagging,0.362427,0.263666,0.950691
5,gb,0.294953,0.203207,0.967544


### Stability

To assess the stability of the data under appropriate perturbations, we first need to define what constitutes an "appropriate" perturbation. In other words, we need to determine what type of data perturbation (e.g., adding random noise or performing sampling) best represents the scenario where the data might have been measured or collected in different ways, and how these results will be applied in the future.

Although the happiness data is not a random sample from a larger population of happiness data across more countries, each data point in the happiness dataset is, to some extent, interchangeable. This means that random sampling techniques would be a reasonable perturbation method. Therefore, we will draw 100 bootstrap samples from the original data.

Additionally, the measurement of Gini index and life expectancy may involve minor measurement errors, even though we do not have a realistic concept of the error range. To truly stress-test the results, we decided to add another perturbation to the data: we will add some random noise to 30% of the `gini_wb_estimate` values. Given that the standard deviation of Gini index is approximately 0.09, we will add or subtract a random number between 0 and 0.045 (i.e., up to half of the standard deviation) to 30% of the `gini_wb_estimate` observations. Similarly, we will add or subtract a random number between 0 and 3.98 to 30% of the `life_expectancy` observations.

Since we will repeat this analysis multiple times, we will write a function that takes the happiness dataset as input and returns its perturbed version.

In [43]:
# write a function that takes the happiness_train_clean data frame and creates a bootstrap sample of the same size
# perturb the gini_wb_estimate column by adding a random number between -0.045 and 0.045 to 30% of the values
# perturb the life_expectancy column by adding a random number between -3.98 and 3.98 to 30% of the values
def perturb_happiness(df):
    # Create a copy of the data frame
    df_copy = df.copy()
    # Generate a random number between -0.045 and 0.045 for 30% of the rows
    sampled_index = df_copy.sample(frac=0.3).index
    df_copy.loc[sampled_index, 'gini_wb_estimate'] += np.random.uniform(-0.045, 0.045, size=sampled_index.size)
    # Generate a random number between -3.98 and 3.98 for 30% of the rows
    sampled_index = df_copy.sample(frac=0.3).index
    df_copy.loc[sampled_index, 'life_expectancy'] += np.random.uniform(-3.98, 3.98, size=sampled_index.size)
    # Conduct bootstrap sample
    df_copy = df_copy.sample(frac=1, replace=True)
    return df_copy

Below we create a list containing the 100 perturbed versions of the training data. 

In [44]:
# create a list of 100 perturbed versions of happiness_train_clean using the perturb_happiness function
perturbed_happiness = [perturb_happiness(happiness_train_clean) for i in range(100)]

We can apply each fit to each of the 100 perturbed training datasets

In [45]:
def fit_models(df, reg=True, fit_social_support_multi=True):
    if fit_social_support_multi==True:
        ls_multi = LinearRegression().fit(X=df[['social_support', 'life_expectancy', 'democratic_quality', 'sd_d_mean_ladder']],
                                                 y=df['happiness'])
    
    rf_fit = RandomForestRegressor().fit(X=df.drop(columns='happiness'), y=df['happiness'])
    bag_fit = BaggingRegressor().fit(X=df.drop(columns='happiness'), y=df['happiness'])
    gb_fit = GradientBoostingRegressor().fit(X=df.drop(columns='happiness'), y=df['happiness'])
    
    # if including the regularized fits, compute them
    if reg:
        
        # standardize predictor variables in df for ridge and lasso
        df_x = df.drop(columns='happiness')
        df_x_std = (df_x - df_x.mean()) / df_x.std()
        df_y = df['happiness']
        
        alphas = np.logspace(-1, 5, 100)
        ridge_cv_scores = []
        for alpha in alphas:
            ridge = Ridge(alpha=alpha)
            ridge_cv = cross_validate(estimator=ridge,
                                    X=df_x_std,
                                    y=df_y,
                                    cv=10,
                                    scoring='neg_root_mean_squared_error')
            ridge_cv_scores.append({'alpha': alpha,
                                    'log_alpha': np.log(alpha),
                                    'test_mse': -np.mean(ridge_cv['test_score'])})
            
        ridge_cv_scores_df = pd.DataFrame(ridge_cv_scores)
        ridge_alpha_min = ridge_cv_scores_df.sort_values(by='test_mse').head(1).alpha.values[0]
        # identify the 1SE value
        mse_se_ridge = ridge_cv_scores_df['test_mse'].std() / np.sqrt(10)
        mse_min_ridge = ridge_cv_scores_df['test_mse'].min()
        ridge_alpha_1se = ridge_cv_scores_df[(ridge_cv_scores_df['test_mse'] <= mse_min_ridge + mse_se_ridge) & 
                                            (ridge_cv_scores_df['test_mse'] >= mse_min_ridge - mse_se_ridge)].sort_values(by='alpha', ascending=False).head(1).alpha.values[0]
        ridge = Ridge(alpha=ridge_alpha_1se).fit(X=df_x_std, y=df_y)
        
        alphas = np.logspace(-2, 7, 100)
        lasso_cv_scores = []
        for alpha in alphas:
            lasso = Lasso(alpha=alpha)
            lasso_cv = cross_validate(estimator=lasso,
                                    X=df_x_std,
                                    y=df_y,
                                    cv=10,
                                    scoring='neg_root_mean_squared_error')
            lasso_cv_scores.append({'alpha': alpha,
                                    'log_alpha': np.log(alpha),
                                    'test_mse': -np.mean(lasso_cv['test_score'])})
        lasso_cv_scores_df = pd.DataFrame(lasso_cv_scores)
        lasso_alpha_min = lasso_cv_scores_df.sort_values(by='test_mse').head(1).alpha.values[0]
        # identify the 1SE value
        mse_se_lasso = lasso_cv_scores_df['test_mse'].std() / np.sqrt(10)
        mse_min_lasso = lasso_cv_scores_df['test_mse'].min()
        lasso_alpha_1se = lasso_cv_scores_df[(lasso_cv_scores_df['test_mse'] <= mse_min_lasso + mse_se_lasso) & 
                                            (lasso_cv_scores_df['test_mse'] >= mse_min_lasso - mse_se_lasso)].sort_values(by='alpha', ascending=False).head(1).alpha.values[0]
        lasso = Lasso(alpha=lasso_alpha_1se).fit(X=df_x_std, y=df_y)
        
        if fit_social_support_multi==True:
            return (ls_multi, ridge, lasso, rf_fit, bag_fit, gb_fit)    
        else:
            return (ridge, lasso, rf_fit, bag_fit, gb_fit)
    
    elif fit_social_support_multi==True:
        return (ls_multi, rf_fit, bag_fit, gb_fit)
    
    else:
        return (rf_fit, bag_fit, gb_fit)

In [46]:
# This code takes a while to run, so we will use the joblib library to parallelize the code
# compute the fits for each perturbed dataset
results = Parallel(n_jobs=-1)(delayed(fit_models)(df) for df in perturbed_happiness)
ls_multi_perturbed, ridge_perturbed, lasso_perturbed,rf_perturbed,bag_perturbed,gb_perturbed = zip(*results)


In [47]:
# compute the predictions on the testidaion set for ls_multi_perturbed, ridge_perturbed, lasso_perturbed, RF_perturbed, Bagging_perturbed and GB_perturbed
ls_multi_test_pred_perturbed = [ls_multi_perturbed[i].predict(X=X_test[['social_support', 'life_expectancy', 'democratic_quality', 'sd_d_mean_ladder']]) for i in range(100)]
ridge_test_pred_perturbed = [ridge_perturbed[i].predict(X=X_test_std) for i in range(100)]
lasso_test_pred_perturbed = [lasso_perturbed[i].predict(X=X_test_std) for i in range(100)]
rf_test_pred_perturbed = [rf_perturbed[i].predict(X=happiness_test_clean.drop(columns='happiness')) for i in range(100)]
bag_test_pred_perturbed = [bag_perturbed[i].predict(X=happiness_test_clean.drop(columns='happiness')) for i in range(100)]
gb_test_pred_perturbed = [gb_perturbed[i].predict(X=happiness_test_clean.drop(columns='happiness')) for i in range(100)]

Let's define a prediction stability plot function

In [48]:
# define a function that takes a list of predictions from ls_perturbed_pred and creates line segment plots for the 
# range of predictions for each id corresponding to the position in each list entry

def plot_prediction_range(pred_list, title=None, sample_index=None):
    if sample_index is None:
        sample_index = list(range(pred_list[0].size))
        
    pred_list = [pred_list[i][sample_index] for i in range(100)]
    pred_list_df = pd.DataFrame(pred_list).T
    pred_list_df['id'] = happiness_test_clean.index[sample_index]
    pred_list_df['true'] = happiness_test_clean['happiness'].values[sample_index]
    pred_list_df = pd.melt(pred_list_df, id_vars=['id','true'], var_name='iter', value_name='pred')
    pred_list_df = pred_list_df.groupby(['id', 'true']).agg({'pred': ['min', 'max']})
    pred_list_df = pred_list_df.reset_index()
    pred_list_df = pred_list_df.set_index('id')

    # plot a series of horizontal line segments for each id where the lines range from the minimum and maximum predicted values on the x-axis and have the true value on the y-axis
    fig = go.Figure()

    for i in pred_list_df.index:
        fig.add_trace(
            go.Scatter(x=[pred_list_df.loc[i, ('pred', 'min')], pred_list_df.loc[i, ('pred', 'max')]],
                        y=[pred_list_df.loc[i, 'true'].values[0], pred_list_df.loc[i, 'true'].values[0]],
                        mode='lines',
                        line={'color': 'black'}, 
                        showlegend=False)
            )
    # add a single diagonal line to the plot
    fig.add_trace(
        go.Scatter(x=[0, 10], y=[0, 10], mode='lines', line={'color': 'black'}, showlegend=False)
    )
        
    fig.update_layout(xaxis_title='Predicted happiness range',
                        yaxis_title='Observed happiness',
                        title=title)
    return fig

And use it to visualize the range of perturbed predictions for 150 test set happiness for each fit. 

In [50]:
test_sample_id = np.random.choice(happiness_test_clean.shape[0], 150, replace=False)
plot_prediction_range(ls_multi_test_pred_perturbed, 'LS (multi)', sample_index=test_sample_id)

In [51]:
plot_prediction_range(ridge_test_pred_perturbed, 'Ridge (1SE)', sample_index=test_sample_id)

In [52]:
plot_prediction_range(lasso_test_pred_perturbed, 'Lasso (1SE)', sample_index=test_sample_id)

In [53]:
plot_prediction_range(rf_test_pred_perturbed, 'RF', sample_index=test_sample_id)

In [54]:
plot_prediction_range(bag_test_pred_perturbed, 'Bagging', sample_index=test_sample_id)

In [55]:
plot_prediction_range(gb_test_pred_perturbed, 'GB', sample_index=test_sample_id)

In [56]:
print(f"LS (multi) average sd: {pd.DataFrame(ls_multi_test_pred_perturbed).std(axis=0).mean()}")
print(f"Ridge (1SE) average sd: {pd.DataFrame(ridge_test_pred_perturbed).std(axis=0).mean()}")
print(f"Lasso (1SE) average sd: {pd.DataFrame(lasso_test_pred_perturbed).std(axis=0).mean()}")
print(f"RF average sd: {pd.DataFrame(rf_test_pred_perturbed).std(axis=0).mean()}")
print(f"Bagging average sd: {pd.DataFrame(bag_test_pred_perturbed).std(axis=0).mean()}")
print(f"GB average sd: {pd.DataFrame(gb_test_pred_perturbed).std(axis=0).mean()}")


LS (multi) average sd: 0.05020975995743011
Ridge (1SE) average sd: 0.06682409134980893
Lasso (1SE) average sd: 0.05370006637304858
RF average sd: 0.12092415955530847
Bagging average sd: 0.16980917281694013
GB average sd: 0.09932031521118004


These findings imply that linear models such as OLS multiple regression, Ridge, and Lasso provide more consistent predictions under data perturbations. In contrast, ensemble methods like Random Forest and especially Bagging exhibit higher variability, possibly due to their reliance on constructing multiple trees, which can be sensitive to changes in the training data. Gradient Boosting falls somewhere in between, showing less variability than Bagging but more than the linear models.